## Cifar 10 Convolutional Neural Network Implementation

This work contains the implementation of two kind of Neural Networks (AlexNet and a Custom model based in LeeNet and AlexNet), validating their capacity to classify the CIFAR10 dataset, a set of images that contains 10 different classes.

In [ ]:
# Pytorch Imports #
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# Torchvision Imports #
import torchvision.datasets as datasets
import torchvision.transforms as T

# matplotlib imports #
import matplotlib.pyplot as plt

# numpy imports #
import numpy as np

### Import Libraries needed for Google Drive mount

In [ ]:
# Library installaation using pip for linux backend of colab#
!pip install -U -q PyDrive

# Import of Libraries from Drive using google collab api #
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google.colab import drive
from google.colab import auth

from oauth2client.client import GoogleCredentials

### <u>Function used to Download data from Drive</u> :

In [ ]:
# Function for downloading especific Data from the Drive folders #
def download_Data(file_list, string_Discriminator):
  # Check inside the folder if is already downlaoded #
  downloaded = []

  for path in os.scandir("/content/"):
    downloaded.append(path.name)

  # Iterate through the files and download them to the data folder if not already downloaded, and are of the givne extension type#
  for files in file_list:
    if (string_Discriminator in files['title']) & (files['title'] not in downloaded):
      print('title: %s, id: %s' % (files['title'], files['id']))
      fname = os.path.join(local_download_path, files['title'])
      print('downloading to {}'.format(fname))
      f_ = drive.CreateFile({'id': files['id']})
      f_.GetContentFile(fname)
      downloaded.append(fname)

### Import Files from Drive

In [ ]:
from google.colab import drive

#Authentication and creation the PyDrive client.
auth.authenticate_user()   #See if credentials are valid
gauth = GoogleAuth()       #Start the authentication of the collab
gauth.credentials = GoogleCredentials.get_application_default()
drive.mount("/content/gdrive")  #Mounts drive in the collab to save the important data
drive = GoogleDrive(gauth) #Finishes the authentication

#Choose a local directory inside the colab to import the data.
local_download_path = os.path.expanduser('/content')

# Try to make the directories in the colab #
try:
  os.makedirs(local_download_path)
except: pass

#Iterate through items using the query syntax for google drive
#https://developers.google.com/drive/v2/web/search-parameters

# Create a file list based on the query syntax searching in our drive folder and download it#
file_list = drive.ListFile(
    {'q': "'1YtW460uumEGz954lHPNUsbwFnjs_tfgW' in parents"}).GetList()

# Download only files from these types from Drive folder 1sqEm5Pvxcg2X2yF2jkZKoojJmSXpysXe #
download_Data(file_list,".py")
download_Data(file_list,".pt")

### Import User Functions

In [ ]:
# User files Imports #
from models.customNet import CNN_custom
from models.alexNet import CNN_AlexNet

from utils.dataset_utils import cifar10_dataset_Generator, verify_data
from utils.general_utils import get_available_devices, set_all_seeds
from utils.matplotlib_utils import plot_figure, plot_figures_grid, plot_minibatch_loss, plot_accuracy_epochs, plot_confusion_matrix, plot_model_outputs
from utils.models_utils import train_cnn, load_model, save_model, compute_total_accuracy, compute_confusion_matrix, get_integrated_gradient, get_occlusion, get_convolutional_layer_weights, get_outputs

## Cuda Verification

In [ ]:
device = get_available_devices()

### Set initial Seed for Neural Network
Allows us to shuffle the model in the same way if we want to get the same initial weights with a Re-Run

In [ ]:
set_all_seeds(0)

### Configure Dataset

We need to prepare the data for the different models training, so the data is transformed using the next parameters:

- Image Dimensions: Original - 32 x 32 x 3, Augmented and cropped - 64 x 64 x 3
- Training Dataset samples - 50000
- Validation Dataset samples - 10000
- Test Dataset samples - 10000
- Minibatch size - 64
- Erasing of pixels - Probability: 0.05, Scale of Erased portion: 0.05, 0.1, Colors Erased: random

Using this augmentation technics we can augmentate the number of images in the dataset, making them different than the original set. This way the networks can train without overfitting the model. In this case the data is not normalized, but the distribution of the clases inside of the dataloaders are evenly distributed. 

In [ ]:
# Dataset Configuration #
MINIBATCH_SIZE = 64
DOWNLOAD = True
LOG = True

# Dataset Transformation Tensor using CIFAR 10 Normalized standar #
transform_cifar = T.Compose([T.ToTensor(),
                             T.Resize((70, 70)),
                             T.RandomCrop((64, 64)),
                             T.RandomErasing(0.05,(0.05,0.1),value="random"),
                             ])

train_data_loader, validation_data_loader, test_data_loader = cifar10_dataset_Generator(transform_cifar,MINIBATCH_SIZE,DOWNLOAD,LOG)

### Verification of Dataloaders

In [ ]:
verify_data(train_data_loader.dataset, validation_data_loader.dataset, test_data_loader.dataset)

### Show Single Image

In [ ]:
plot_figure(train_data_loader)

### Show Multiple Images

In [ ]:
plot_figures_grid(train_data_loader)

### AlexNet Paper Model

References

[1] Krizhevsky, Alex, Ilya Sutskever, and Geoffrey E. Hinton. "Imagenet classification with deep convolutional neural networks." In Advances in Neural Information Processing Systems, pp. 1097-1105. 2012.

- This model has an architecture with maxpooling and dropout, with the next hyperparameters.
- Learning Rate: 0.0001
- Epochs: 20
- Minibatch: 64
- Optimizer: Adams

### Neural Network AlexNet Model Implementation

In [ ]:
# Hyperparameters Definitions
LEARNING_RATE = 0.0001
NUM_EPOCHS = 20

# Architecture
NUM_CLASSES = 10

# Model extension save
Model_NAME = "alexNet.pt"

# Create model using AlexNet class
alexNet_model = CNN_AlexNet(NUM_CLASSES)

# Send Device to GPU  if available
alexNet_model.to(device)

# Optimizer implementation - Adam
optimizer = torch.optim.Adam(alexNet_model.parameters(), lr=LEARNING_RATE)

### Training of Network

In [ ]:
logger = train_cnn(NUM_EPOCHS,alexNet_model,optimizer,device,train_data_loader,validation_data_loader)

### Save Network Model in drive

In [ ]:
save_model(alexNet_model,Model_NAME)
save_model(optimizer, "optimizer_" + Model_NAME)

### Evaluation of Model

In [ ]:
plot_minibatch_loss(logger,"train_loss_batch")

In [ ]:
plot_accuracy_epochs(logger,NUM_EPOCHS,["train_accuracy_epoch","validation_accuracy_epoch"])

### Graphs Explanations

- We can see that the loss function has an uneven behavior, but the average is an stable descending curve, meaning the model is accurately training.
- In case of the accuracy per epoch, we can see it the training start ramping up in the first epochs and then slows down, this could be attributed to the complexity of the network for the dataset scale, also the dropout in the classification linear part is high, so it may start to saturate the model to the point it does not train anymore. 

### Neural Network Custom Model Implementation

- This model has an architecture with maxpooling and dropout, with the next hyperparameters.
- Learning Rate: 0.00038
- Epochs: 20
- Minibatch: 64
- Optimizer: Adams

In [ ]:
# Hyperparameters Definitions
LEARNING_RATE = 0.00038
NUM_EPOCHS = 20

# Architecture
NUM_CLASSES = 10

# Model extension save
Model_NAME = "custom_net.pt"

# Create model using AlexNet class
custom_model = CNN_custom(NUM_CLASSES)

custom_model.to(device)

# Optimizer implementation - Adam
optimizer = torch.optim.Adam(custom_model.parameters(), lr=LEARNING_RATE)

### Evaluation of Model

In [ ]:
logger = train_cnn(NUM_EPOCHS,custom_model,optimizer,device,train_data_loader,validation_data_loader)

### Save Network Model in drive

In [ ]:
save_model(custom_model, Model_NAME)
save_model(optimizer, "optimizer_" + Model_NAME)

### Evaluation of Model

In [ ]:
plot_minibatch_loss(logger,"train_loss_batch")

In [ ]:
plot_accuracy_epochs(logger,NUM_EPOCHS,["train_accuracy_epoch","validation_accuracy_epoch"])

### Graphs Explanations

- We can see that the loss function has an uneven behavior, but the average is an stable descending curve, meaning the model is accurately training.
- In case of the accuracy per epoch, we can see it the training start ramping up in the first epochs and then slows down, but its not overfitting to the dataset because the validation have not reached a maximum value. We can say that the objective of getting more than 70% percent of accuracy have been achieved.